In [3]:
!pip install spacy-langdetect
!pip install language-detector
!pip install symspellpy
!pip install sentence-transformers
!pip install --upgrade gensim
!pip install pyLDAvis3
!pip install stop-words
!python -m pip install -U pyLDAvis
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('words')
nltk.download('punkt')

  Using cached https://files.pythonhosted.org/packages/19/22/aff234f4a841f8999e68a7a94bdd4b60b4cebcfeca5d67d61cd08c9179de/transformers-3.3.1-py3-none-any.whl
  Found existing installation: transformers 2.2.2
    Uninstalling transformers-2.2.2:
      Successfully uninstalled transformers-2.2.2
Requirement already up-to-date: gensim in /usr/local/lib/python3.6/dist-packages (3.8.3)
ERROR: Could not find a version that satisfies the requirement pyLDAvis3 (from versions: none)
ERROR: No matching distribution found for pyLDAvis3
Requirement already up-to-date: pyLDAvis in /usr/local/lib/python3.6/dist-packages (2.1.2)
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]

True

In [4]:
#importing all dependencies.
import os
import json
import pandas as pd
from tqdm import tqdm
import numpy as np
from nltk.corpus import wordnet
import re
import matplotlib.pyplot as plt
from nltk.corpus import stopwords 
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# Topic Modeling with BERT, LDA, and Clustering
#### Latent Dirichlet Allocation (LDA) probabilistic topic assignment and pre-trained sentence embeddings from BERT/RoBERTa

## Model Deep Dive

The author used: 

* LDA for probabilistic topic assignment vector.
* Bert for sentence embedding vector.

1. Concatenated both LDA and Bert vectors with a weight hyperparameter to balance the relative importance of information from each source.
2. Used autoencoder to learn a lower dimensional latent space representation of the concatenated vector.

* The assumption is that the concatendate vector shoul have a manifold shaep in the high dimensional space. 
* USed clustering on the latent space representations to get topics. 



![Contextual Topic Identification model design](https://miro.medium.com/max/1410/1*OKCYnB-JbGq1NDwNSKd5Zw.png)

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
df= pd.read_excel(r'/content/drive/My Drive/CFPB LDA/LDA Output Document to topic.xlsx')
data = df['Cleansed_Text']

In [7]:
'''import pandas as pd
df1= pd.read_csv(r'/content/drive/My Drive/CFPB LDA/Mortgage_complaints_text_test.csv')
df2= pd.read_excel(r'/content/drive/My Drive/CFPB LDA/CFPB_text_vs_cnsd_text_20201013_08_05_41.xlsx')
df1 = df1.rename(columns={'Complaint ID':'Complaint_ID'})
print(df1.columns)
print(df2.columns)
df = pd.merge(df2[['Complaint_ID', 'Complain_text', 'Modified_text']], df1[['Complaint_ID','Issue','Text_Length']], on=['Complaint_ID'])
df["Modified_text"] = df["Modified_text"].replace(np.nan, '', regex=True)'''
#df.info()

'import pandas as pd\ndf1= pd.read_csv(r\'/content/drive/My Drive/CFPB LDA/Mortgage_complaints_text_test.csv\')\ndf2= pd.read_excel(r\'/content/drive/My Drive/CFPB LDA/CFPB_text_vs_cnsd_text_20201013_08_05_41.xlsx\')\ndf1 = df1.rename(columns={\'Complaint ID\':\'Complaint_ID\'})\nprint(df1.columns)\nprint(df2.columns)\ndf = pd.merge(df2[[\'Complaint_ID\', \'Complain_text\', \'Modified_text\']], df1[[\'Complaint_ID\',\'Issue\',\'Text_Length\']], on=[\'Complaint_ID\'])\ndf["Modified_text"] = df["Modified_text"].replace(np.nan, \'\', regex=True)'

# summa – textrank

* Text summarization

In [8]:
!pip install summa
from summa import summarizer

In [9]:
text1 = df['Original_Text'][0]
text2 = df['Cleansed_Text'][0]
print('Orginal Text: ',text1)
print('Summarize Text: ',summarizer.summarize(text1))
print('Orginal Text: ',text2)
print('Summarize Text: ',summarizer.summarize(text2))

Orginal Text:  Greetings, Nationstar mortgage contacted me saying that my association no longer had valid flood insurance. I checked with my association and it turns out we no longer needed flood insurance for my condo. I got the required " Standard Flood Hazard Determination '' document from my condo stating that the condo was no longer in a flood zone. I uploaded the document per Nationstar instructions and also faxed it. A month letter I got another letter. I called and they said they did get the documents but they were backed up and would fix it. I received at least XXXX more letters and called XXXX more times. Each time I was told this would be fixed quickly. Finally they took out an escrow to pay for the flood insurance I did n't need. I called and they showed that part of the insurance had been refunded but not all. Again, they said they would fix.
Summarize Text:  I checked with my association and it turns out we no longer needed flood insurance for my condo.
I called and they 

* Keyword extraction

In [10]:
from summa import keywords

In [11]:
print('Orginal Text: ',text1)
print('Keywords of Text: ',keywords.keywords(text1))
print('Orginal Text: ',text2)
print('Keywords of Text: ',keywords.keywords(text2))

Orginal Text:  Greetings, Nationstar mortgage contacted me saying that my association no longer had valid flood insurance. I checked with my association and it turns out we no longer needed flood insurance for my condo. I got the required " Standard Flood Hazard Determination '' document from my condo stating that the condo was no longer in a flood zone. I uploaded the document per Nationstar instructions and also faxed it. A month letter I got another letter. I called and they said they did get the documents but they were backed up and would fix it. I received at least XXXX more letters and called XXXX more times. Each time I was told this would be fixed quickly. Finally they took out an escrow to pay for the flood insurance I did n't need. I called and they showed that part of the insurance had been refunded but not all. Again, they said they would fix.
Keywords of Text:  nationstar mortgage contacted
instructions
greetings
Orginal Text:  greeting mortgage contact me say that my asso

# Bert Extractive Summarizer

In [12]:
!pip install bert-extractive-summarizer

In [13]:
!pip install spacy==2.1.3
!pip install transformers==2.2.2
!pip install neuralcoref

!python -m spacy download en_core_web_md

  Using cached https://files.pythonhosted.org/packages/d1/08/4a6768ca1a7a4fa37e5ee08077c5d02b8d83876bd36caa5fc24d98992ac2/transformers-2.2.2-py3-none-any.whl
ERROR: sentence-transformers 0.3.8 has requirement transformers<3.4.0,>=3.1.0, but you'll have transformers 2.2.2 which is incompatible.
  Found existing installation: transformers 3.3.1
    Uninstalling transformers-3.3.1:
      Successfully uninstalled transformers-3.3.1
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


#### Simple Example

In [14]:
from summarizer import Summarizer

model = Summarizer()
print('Orginal Text: ',text1)
print('Summarize Text: ',model(text1))
print('Orginal Text: ',text2)
print('Summarize Text: ',model(text2))

#Specifying number of sentences
result = model(text1, ratio=0.2)  # Specified with ratio
print('Summarize Text: ',model(text1))
result = model(text1, num_sentences=3)
print('Summarize Text: ',model(result))

KeyboardInterrupt: ignored

In [ ]:
result = model(text1, min_length=60)
full = ''.join(result)
print(full)

# PyTLDR: Automatic Text Summarization in Python

In [ ]:
!pip install git+https://github.com/jaijuneja/PyTLDR.git

In [ ]:
from pytldr.summarize import TextRankSummarizer
from pytldr.nlp import Tokenizer

tokenizer = Tokenizer('english')
summarizer = TextRankSummarizer(tokenizer)

# If you don't specify a tokenizer when intiializing a summarizer then the
# English tokenizer will be used by default
summarizer = TextRankSummarizer()  # English tokenizer used

# This object creates a summary using the summarize method:
# e.g. summarizer.summarize(text, length=5, weighting='frequency', norm=None)

# The length parameter specifies the length of the summary, either as a
# number of sentences, or a percentage of the original text

# The summarizer can take as input...
# 1. A string:
summary = summarizer.summarize(text1, length=4)
print(summary)

# Latent Semantic Analysis (LSA) Summarization

In [ ]:
from pytldr.summarize import LsaSummarizer, LsaOzsoy, LsaSteinberger

summarizer = LsaOzsoy()
summarizer = LsaSteinberger()
summarizer = LsaSummarizer()  # This is identical to the LsaOzsoy object

summary = summarizer.summarize(
    text, topics=4, length=5, binary_matrix=True, topic_sigma_threshold=0.5
)
print(summary)

# Implementing Text Summarization in Python using Keras

In [ ]:
from attention import AttentionLayer

In [ ]:
import numpy as np  
import pandas as pd 
import re           
from bs4 import BeautifulSoup 
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords   
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import warnings
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")

## Texts summarizing with the help of Spacy

In [ ]:
#!pip install spacy
#!python -m spacy download en_core_web_lg
import spacy
nlp = spacy.load('en_core_web_lg')

In [17]:
import heapq
# this is function for text summarization

def generate_summary(text_without_removing_dot, cleaned_text):
    sample_text = text_without_removing_dot
    doc = nlp(sample_text)
    sentence_list=[]
    for idx, sentence in enumerate(doc.sents): # we are using spacy for sentence tokenization
        sentence_list.append(re.sub(r'[^\w\s]','',str(sentence)))

    stopwords = nltk.corpus.stopwords.words('english')

    word_frequencies = {}  
    for word in nltk.word_tokenize(cleaned_text):  
        if word not in stopwords:
            if word not in word_frequencies.keys():
                word_frequencies[word] = 1
            else:
                word_frequencies[word] += 1


    maximum_frequncy = max(word_frequencies.values())

    for word in word_frequencies.keys():  
        word_frequencies[word] = (word_frequencies[word]/maximum_frequncy)


    sentence_scores = {}  
    for sent in sentence_list:  
        for word in nltk.word_tokenize(sent.lower()):
            if word in word_frequencies.keys():
                if len(sent.split(' ')) < 30:
                    if sent not in sentence_scores.keys():
                        sentence_scores[sent] = word_frequencies[word]
                    else:
                        sentence_scores[sent] += word_frequencies[word]


    summary_sentences = heapq.nlargest(7, sentence_scores, key=sentence_scores.get)

    summary = ' '.join(summary_sentences)
    print("Original Text::::::::::::\n")
    print(text_without_removing_dot)
    print('\n\nSummarized text::::::::\n')
    print(summary)  

In [19]:
generate_summary(df['Original_Text'][0], df['Original_Text'][0])

Original Text::::::::::::

Greetings, Nationstar mortgage contacted me saying that my association no longer had valid flood insurance. I checked with my association and it turns out we no longer needed flood insurance for my condo. I got the required " Standard Flood Hazard Determination '' document from my condo stating that the condo was no longer in a flood zone. I uploaded the document per Nationstar instructions and also faxed it. A month letter I got another letter. I called and they said they did get the documents but they were backed up and would fix it. I received at least XXXX more letters and called XXXX more times. Each time I was told this would be fixed quickly. Finally they took out an escrow to pay for the flood insurance I did n't need. I called and they showed that part of the insurance had been refunded but not all. Again, they said they would fix.


Summarized text::::::::

I got the required  Standard Flood Hazard Determination  document from my condo stating that 

## Upload Data

**Data pipeline (from development to deployment**)

![Data pipeline (from development to deployment)](https://miro.medium.com/max/1348/1*Cdp4y1tfMxqoj96o6lUdFg.png)


Source:Shoa 

In [ ]:
#load library
import os
import pandas as pd
import numpy as np
import gensim
from gensim.utils import simple_preprocess
from gensim import corpora, models
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *

import datetime
import time
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import nltk

## Utils

In [ ]:
from collections import Counter
from sklearn.metrics import silhouette_score
import umap
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from gensim.models.coherencemodel import CoherenceModel
import numpy as np
import os


def get_topic_words(token_lists, labels, k=None):
    """
    get top words within each topic from clustering results
    """
    if k is None:
        k = len(np.unique(labels))
    topics = ['' for _ in range(k)]
    for i, c in enumerate(token_lists):
        topics[labels[i]] += (' ' + ' '.join(c))
    word_counts = list(map(lambda x: Counter(x.split()).items(), topics))
    # get sorted word counts
    word_counts = list(map(lambda x: sorted(x, key=lambda x: x[1], reverse=True), word_counts))
    # get topics
    topics = list(map(lambda x: list(map(lambda x: x[0], x[:10])), word_counts))

    return topics

def get_coherence(model, token_lists, measure='c_v'):
    """
    Get model coherence from gensim.models.coherencemodel
    :param model: Topic_Model object
    :param token_lists: token lists of docs
    :param topics: topics as top words
    :param measure: coherence metrics
    :return: coherence score
    """
    if model.method == 'LDA':
        cm = CoherenceModel(model=model.ldamodel, texts=token_lists, corpus=model.corpus, dictionary=model.dictionary,
                            coherence=measure)
    else:
        topics = get_topic_words(token_lists, model.cluster_model.labels_)
        cm = CoherenceModel(topics=topics, texts=token_lists, corpus=model.corpus, dictionary=model.dictionary,
                            coherence=measure)
    return cm.get_coherence()

def get_silhouette(model):
    """
    Get silhouette score from model
    :param model: Topic_Model object
    :return: silhouette score
    """
    if model.method == 'LDA':
        return
    lbs = model.cluster_model.labels_
    vec = model.vec[model.method]
    return silhouette_score(vec, lbs)

def plot_proj(embedding, lbs):
    """
    Plot UMAP embeddings
    :param embedding: UMAP (or other) embeddings
    :param lbs: labels
    """
    n = len(embedding)
    counter = Counter(lbs)
    for i in range(len(np.unique(lbs))):
        plt.plot(embedding[:, 0][lbs == i], embedding[:, 1][lbs == i], '.', alpha=0.5,
                 label='cluster {}: {:.2f}%'.format(i, counter[i] / n * 100))
    plt.legend(loc = 'best')
    plt.grid(color ='grey', linestyle='-',linewidth = 0.25)


def visualize(model):
    """
    Visualize the result for the topic model by 2D embedding (UMAP)
    :param model: Topic_Model object
    """
    if model.method == 'LDA':
        return
    reducer = umap.UMAP()
    print('Calculating UMAP projection ...')
    vec_umap = reducer.fit_transform(model.vec[model.method])
    print('Calculating UMAP projection. Done!')
    plot_proj(vec_umap, model.cluster_model.labels_)
    dr = '/content/drive/My Drive/CFPB LDA/{}'.format(model.method, model.id)
    if not os.path.exists(dr):
        os.makedirs(dr)
    plt.savefig('/content/drive/My Drive/CFPB LDA/')

def get_wordcloud(model, token_lists, topic):
    """
    Get word cloud of each topic from fitted model
    :param model: Topic_Model object
    :param sentences: preprocessed sentences from docs
    """
    if model.method == 'LDA':
        return
    print('Getting wordcloud for topic {} ...'.format(topic))
    lbs = model.cluster_model.labels_
    tokens = ' '.join([' '.join(_) for _ in np.array(token_lists)[lbs == topic]])

    wordcloud = WordCloud(width=800, height=560,
                          background_color='white', collocations=False,
                          min_font_size=10).generate(tokens)

    # plot the WordCloud image
    plt.figure(figsize=(8, 5.6), facecolor=None)
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.tight_layout(pad=0)
    dr = '/kaggle/working/{}/{}'.format(model.method, model.id)
    if not os.path.exists(dr):
        os.makedirs(dr)
    plt.savefig('/kaggle/working' + '/Topic' + str(topic) + '_wordcloud')
    print('Getting wordcloud for topic {}. Done!'.format(topic))

## Preprocessing 

In [ ]:
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
import re
import nltk
from nltk.tokenize import word_tokenize
from language_detector import detect_language

import pkg_resources
from symspellpy import SymSpell, Verbosity

sym_spell = SymSpell(max_dictionary_edit_distance=3, prefix_length=7)
dictionary_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_dictionary_en_82_765.txt")
if sym_spell.word_count:
    pass
else:
    sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)


###################################
#### sentence level preprocess ####
###################################

# lowercase + base filter
# some basic normalization
def f_base(s):
    """
    :param s: string to be processed
    :return: processed string: see comments in the source code for more info
    """
    '''# normalization 1: xxxThis is a --> xxx. This is a (missing delimiter)
    s = re.sub(r'([a-z])([A-Z])', r'\1\. \2', s)  # before lower case
    # normalization 2: lower case
    s = s.lower()
    # normalization 3: "&gt", "&lt"
    s = re.sub(r'&gt|&lt', ' ', s)
    # normalization 4: letter repetition (if more than 2)
    s = re.sub(r'([a-z])\1{2,}', r'\1', s)
    # normalization 5: non-word repetition (if more than 1)
    s = re.sub(r'([\W+])\1{1,}', r'\1', s)
    # normalization 6: string * as delimiter
    s = re.sub(r'\*|\W\*|\*\W', '. ', s)
    # normalization 7: stuff in parenthesis, assumed to be less informal
    s = re.sub(r'\(.*?\)', '. ', s)
    # normalization 8: xxx[?!]. -- > xxx.
    s = re.sub(r'\W+?\.', '.', s)
    # normalization 9: [.?!] --> [.?!] xxx
    s = re.sub(r'(\.|\?|!)(\w)', r'\1 \2', s)
    # normalization 10: ' ing ', noise text
    s = re.sub(r' ing ', ' ', s)
    # normalization 11: noise text
    s = re.sub(r'product received for free[.| ]', ' ', s)
    # normalization 12: phrase repetition
    s = re.sub(r'(.{2,}?)\1{1,}', r'\1', s)'''

    #return s.strip()
    return s


# language detection
def f_lan(s):
    """
    :param s: string to be processed
    :return: boolean (s is English)
    """

    # some reviews are actually english but biased toward french
    return 1


###############################
#### word level preprocess ####
###############################

# filtering out punctuations and numbers
def f_punct(w_list):
    """
    :param w_list: word list to be processed
    :return: w_list with punct and number filter out
    """
    return [word for word in w_list if word.isalpha()]


# selecting nouns
def f_noun(w_list):
    """
    :param w_list: word list to be processed
    :return: w_list with only nouns selected
    """
    return [word for (word, pos) in nltk.pos_tag(w_list) if pos[:2] == 'NN']


# typo correction
def f_typo(w_list):
    """
    :param w_list: word list to be processed
    :return: w_list with typo fixed by symspell. words with no match up will be dropped
    """
    w_list_fixed = []
    for word in w_list:
        suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=3)
        if suggestions:
            w_list_fixed.append(suggestions[0].term)
        else:
            pass
            # do word segmentation, deprecated for inefficiency
            # w_seg = sym_spell.word_segmentation(phrase=word)
            # w_list_fixed.extend(w_seg.corrected_string.split())
    return w_list_fixed


# stemming if doing word-wise
p_stemmer = PorterStemmer()


def f_stem(w_list):
    """
    :param w_list: word list to be processed
    :return: w_list with stemming
    """
    return [p_stemmer.stem(word) for word in w_list]


# filtering out stop words
# create English stop words list

stop_words = (list(
    set(get_stop_words('en'))
    |set(get_stop_words('es'))
    |set(get_stop_words('de'))
    |set(get_stop_words('it'))
    |set(get_stop_words('ca'))
    #|set(get_stop_words('cy'))
    |set(get_stop_words('pt'))
    #|set(get_stop_words('tl'))
    |set(get_stop_words('pl'))
    #|set(get_stop_words('et'))
    |set(get_stop_words('da'))
    |set(get_stop_words('ru'))
    #|set(get_stop_words('so'))
    |set(get_stop_words('sv'))
    |set(get_stop_words('sk'))
    #|set(get_stop_words('cs'))
    |set(get_stop_words('nl'))
    #|set(get_stop_words('sl'))
    #|set(get_stop_words('no'))
    #|set(get_stop_words('zh-cn'))
))

def f_stopw(w_list):
    """
    filtering out stop words
    """
    return [word for word in w_list if word not in stop_words]


def preprocess_sent(rw):
    """
    Get sentence level preprocessed data from raw review texts
    :param rw: review to be processed
    :return: sentence level pre-processed review
    """
    s = f_base(rw)
    if not f_lan(s):
        return None
    return s


def preprocess_word(s):
    """
    Get word level preprocessed data from preprocessed sentences
    including: remove punctuation, select noun, fix typo, stem, stop_words
    :param s: sentence to be processed
    :return: word level pre-processed review
    """
    if not s:
        return None
    w_list = word_tokenize(s)
    w_list = f_punct(w_list)
    w_list = f_noun(w_list)
    w_list = f_typo(w_list)
    w_list = f_stem(w_list)
    w_list = f_stopw(w_list)

    return w_list

## Autoencoder

In [ ]:
import keras
from keras.layers import Input, Dense
from keras.models import Model
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')


class Autoencoder:
    """
    Autoencoder for learning latent space representation
    architecture simplified for only one hidden layer
    """

    def __init__(self, latent_dim=32, activation='relu', epochs=200, batch_size=128):
        self.latent_dim = latent_dim
        self.activation = activation
        self.epochs = epochs
        self.batch_size = batch_size
        self.autoencoder = None
        self.encoder = None
        self.decoder = None
        self.his = None

    def _compile(self, input_dim):
        """
        compile the computational graph
        """
        input_vec = Input(shape=(input_dim,))
        encoded = Dense(self.latent_dim, activation=self.activation)(input_vec)
        decoded = Dense(input_dim, activation=self.activation)(encoded)
        self.autoencoder = Model(input_vec, decoded)
        self.encoder = Model(input_vec, encoded)
        encoded_input = Input(shape=(self.latent_dim,))
        decoder_layer = self.autoencoder.layers[-1]
        self.decoder = Model(encoded_input, self.autoencoder.layers[-1](encoded_input))
        self.autoencoder.compile(optimizer='adam', loss=keras.losses.mean_squared_error)

    def fit(self, X):
        if not self.autoencoder:
            self._compile(X.shape[1])
        X_train, X_test = train_test_split(X)
        self.his = self.autoencoder.fit(X_train, X_train,
                                        epochs=200,
                                        batch_size=128,
                                        shuffle=True,
                                        validation_data=(X_test, X_test), verbose=0)

## Model

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from gensim import corpora
import gensim
import numpy as np
#from Autoencoder import *
#from preprocess import *
from datetime import datetime


def preprocess(docs, samp_size=None):
    """
    Preprocess the data
    """
    if not samp_size:
        samp_size = 100

    print('Preprocessing raw texts ...')
    n_docs = len(docs)
    sentences = []  # sentence level preprocessed
    token_lists = []  # word level preprocessed
    idx_in = []  # index of sample selected
    #     samp = list(range(100))
    samp = np.random.choice(n_docs, samp_size)
    for i, idx in enumerate(samp):
        sentence = preprocess_sent(docs[idx])
        token_list = preprocess_word(sentence)
        if token_list:
            idx_in.append(idx)
            sentences.append(sentence)
            token_lists.append(token_list)
        print('{} %'.format(str(np.round((i + 1) / len(samp) * 100, 2))), end='\r')
    print('Preprocessing raw texts. Done!')
    return sentences, token_lists, idx_in


# define model object
class Topic_Model:
    def __init__(self, k=11, method='TFIDF'):
        """
        :param k: number of topics
        :param method: method chosen for the topic model
        """
        if method not in {'TFIDF', 'LDA', 'BERT', 'LDA_BERT'}:
            raise Exception('Invalid method!')
        self.k = k
        self.dictionary = None
        self.corpus = None
        self.np_corpus = None
        self.corpus_new = None
        self.del_ids = None
        #         self.stopwords = None
        self.cluster_model = None
        self.ldamodel = None
        self.vec = {}
        self.gamma = 15  # parameter for reletive importance of lda
        self.method = method
        self.AE = None
        self.id = method + '_' + datetime.now().strftime("%Y_%m_%d_%H_%M_%S")

    def vectorize(self, sentences, token_lists, method=None):
        """
        Get vecotr representations from selected methods
        """
        # Default method
        if method is None:
            method = self.method
        # turn tokenized documents into a id <-> term dictionary
        self.dictionary = corpora.Dictionary(token_lists)      

        #Remove unneccessary keywords from token list of corpus. First identify the keywords and then remove them from dictionary and respectively on the corpus vector            
        #rmv_words_corpus = ['provide','account','any','them','we','payment','or','company','dispute','you','request','by','an','our','your','will','consumer','due','send','which','over','receive','inform','show','never','add','their','please','owe','now','what','proof','state','amount','late','because','check','since','ask','apply','back','take','mine','notify','only','belong_me','can','proper','there','mail','know','sign','default','eviction','name','additional','debt','accuracy','behind','allege','subsection','regular','manager','instrument_bear','subparagraph','render','mother','content','accordance_with','landlord','door','data_furnisher','exact_same','description','second','completeness','shall','base_upon','absent','ratio','justify','sister','reasonably','operate','see_attachment','regularly','threatening','marry','stopped','bury','morning','pad','operator','displace','ship','mediate','say','contact','letter','need','insurance','so','credit','out','she','also','date','bank','give','up','just','change','increase','recent','shortage','how_much','partial','flood','renewal','periodic','reminder','apology','spread','transition','gap','rudely','flood_zone','midst','maturity','master_policy','document','file','who','case','modification','new','attorney','still','record','correct','case_number','against','action','client','lie','trustee','attached','dual','recast','caliber','favor','relevant','entry','consequence','nominee','execution','character','days','clock','herein','united','pursuit','chat','notwithstanding','concrete','yours','willful','undo','inaction','patiently','hate','tax','try','address','could','work','process','at','copy','free','inquiry','deny','balance','off','all','foreclose','make','into','investigation','get','husband','number','fraud','personal','able','rate','want','way','access','submit','plan','month','form','assist','act','offer','county','initiate','rent','trust','site','lower','discover','live_at','understand','view','privacy','consent','employee','additionally','fill_out','best','residence','connect','obvious','pin','insert','conference','eligible','go','defer','back_onto','retrieve','annual','motion','expose','unwanted','carry','crime','taxpayer','vehicle','exempt','divide','racketeering','jumbo','imply','technical','reassess','bond','subpoena','hedge','gon_na','husband_die','star','scary','proven','bubble','spirit','excite','earner','margin','he','fee','but','again','close','application','time','bureau','these','help','transfer','list','after','keep','lender','even','more','appraisal','representative','attach','him','per','return','order','his','nothing','none','deed','particular','packet','pa','comment','age','bogus','identification','child','draft','real','waive','might','sent_certificate','hire','legitimately','treat','together','contractual','sum','stole','desist','world','mi','harassment','direction','imagine','counselor','corporate','priority','business_practice','sheriff','remodification','reassign','arm','rush','combine','farm','faster','ethic','annuity','stall_tactic','declination','click','administer','offset','teacher','folder','unlike','dissatisfy','naturally','tell','when','card','should','would','score','service','escrow','negative','one','bill','find','continue','issue','original','refuse','multiple','response','term','value','private','schedule','refund','power','release','post','reflect','average','press','front','rep','meanwhile','watch','karma','user','frustration','certificate','intentionally','accident','enforce','deduct','evidently','length','percent','green','time_fashion','newly','manual','ie','float','pro','anxious','frankly','assessed','pay','sever','verify','incorrect','creditor','open','false','delete','use','call','regard','phone','another','previous','speak','total','current','satisfy','estimate','submission','understanding','town','stuff','enclose','generic','photo','electronic','sincerely','advance','specify','rather_than','doc','pass','exemption','scan','hud','disburse','shortfall','wrongful','accumulate','minus','gift','realistic','contention','recur','column','girl','flip','sloppy','caught','continuance','buck','house','sell','buy','about','through','her','loan','update','right','same','start','place','own','result','resolve','down','title','low','away','happen','spoke','price','benefit','represent','firm','unit','got','messenger','duplex','man','neighbor','vacant','incident','per_month','incur','urgent','opinion','reinsert','extension','prefer','friend','borrow','perfect','sheet','street','water','brother','sometime','reward','differ','belonging','occupy','floor','yard','tree','ex_husband','marketing','dog','score_suffer','wide','liar','shape','predator','driveway','drug','redemption','cash_key','wow','heat','kickback','recorder','item','inaccurate','charge','interest','money','fund','law','immediately','court','principle','put','must','furnish','policy','wrong','lien','source','figure','treatment','challenge','contractor','physical','any_kind','certification','certify','suspense','draw','relief','verifiable','deem','statue','adjudication','wit','disturb','entirely','nonexistent','eye','frozen','affair','inspector','auto','fay','revenue','storm','forced','teller','index','enough_cover','recalculate','payee','septic','occurrence','unhelpful','manually','spoken','review','require','approve','report','borrower','advise','within','delay','rule','guideline','appraiser','builder','accurately','tip','evade','chart','exclude','duration','directive','conditional','origin','noncompliance','rebate','remit','procure','reform','questionnaire','accent','foreclosure','short_sale','complain','complete','notice','other','income','sale','office','trial','appeal','decision','conduct','specialist','exhibit','forbearance','mediation','forge','eligibility','exercise','article','blank','hereby','format','foreign','harp','limitation','civil_code','commissioner','writer','log','facially_complete']
        rmv_words_corpus = ['inform','receive','mortgage','number','send','work','short','make','mother','account','contact','tell','proper','request','customer','wife','say','company','family','refinance','basically','date','value','unit','provide','check','job','record','continue','submit','roof','apply','complain','trust','new','remove','approve','land','try','increase','ask','husband','sign','high','right','correct','program','need','situation','term','chapter','line','search','bank','shortage','personal','cause','original','buy','trial','want','replace','case','loss','low','demand','negative','just','flood','representative','hardship','attempt','dispute','apartment','live','child','action','current','history','know','commitment','homeowner','response','speak','life','assignment','refuse','perform','analysis','owe','feel','complete','add','hire','county','additional','borrower','use','pull','divorce','option','require','start','place','write','question','copy','consumer','final']
        # assume the word 'b' is to be deleted, put its id in a variable
        self.del_ids = [k for k,v in self.dictionary.items() if v in rmv_words_corpus]

        # remove unwanted word ids from the dictionary in place
        self.dictionary.filter_tokens(bad_ids=self.del_ids)
        # convert tokenized documents into a document-term matrix
        self.corpus = [self.dictionary.doc2bow(text) for text in token_lists]
        gensim.corpora.MmCorpus.serialize('Mortgage_CorpusAll.mm', self.corpus)        
        '''self.corpus = gensim.corpora.MmCorpus('Mortgage_CorpusAll.mm')
        # convert corpus to a dense array, transpose because by default documents would be columns
        self.np_corpus = gensim.matutils.corpus2dense(self.corpus, self.corpus.num_terms, num_docs=self.corpus.num_docs).T
        # delete columns for specified tokens, transpose back afterwards
        self.np_corpus = np.delete(self.np_corpus, self.del_ids, 1).T
        # convert array to corpus
        self.corpus_new = gensim.matutils.Dense2Corpus(self.np_corpus)'''
        

        if method == 'TFIDF':
            print('Getting vector representations for TF-IDF ...')
            tfidf = TfidfVectorizer()
            vec = tfidf.fit_transform(sentences)
            print('Getting vector representations for TF-IDF. Done!')
            return vec

        elif method == 'LDA':
            print('Getting vector representations for LDA ...')
            if not self.ldamodel:
                self.ldamodel = gensim.models.ldamodel.LdaModel(self.corpus, num_topics=self.k, id2word=self.dictionary,
                                                                passes=20)
                self.ldamodel.save('lda_train4.model')

            def get_vec_lda(model, corpus, k):
                """
                Get the LDA vector representation (probabilistic topic assignments for all documents)
                :return: vec_lda with dimension: (n_doc * n_topic)
                """
                n_doc = len(corpus)
                vec_lda = np.zeros((n_doc, k))
                for i in range(n_doc):
                    # get the distribution for the i-th document in corpus
                    for topic, prob in model.get_document_topics(corpus[i]):
                        vec_lda[i, topic] = prob                
                return vec_lda

            vec = get_vec_lda(self.ldamodel, self.corpus, self.k)
            pickle.dump(vec, open("Mortgage_train_vec.pkl", "wb" ))
            print('Getting vector representations for LDA. Done!')
            return vec

        elif method == 'BERT':
            '''## Load required packages for FinBERT
            from transformers import AutoTokenizer, AutoModelForSequenceClassification
            import torch
            import numpy as np


            %reload_ext watermark
            %watermark -v -p numpy,torch,transformers
            print('Getting vector representations for BERT ...')
            tokenizer = AutoTokenizer.from_pretrained("ipuneetrathore/bert-base-cased-finetuned-finBERT")
            model = AutoModelForSequenceClassification.from_pretrained("ipuneetrathore/bert-base-cased-finetuned-finBERT")'''
            print('Getting vector representations for BERT ...')
            from sentence_transformers import SentenceTransformer
            model = SentenceTransformer('bert-base-nli-max-tokens')
            vec = np.array(model.encode(sentences, show_progress_bar=True))
            print('Getting vector representations for BERT. Done!')
            return vec
            vec = np.array(model.encode(sentences, show_progress_bar=True))
            print('Getting vector representations for BERT. Done!')
            return vec

             
        elif method == 'LDA_BERT':
        #else:
            vec_lda = self.vectorize(sentences, token_lists, method='LDA')
            vec_bert = self.vectorize(sentences, token_lists, method='BERT')
            vec_ldabert = np.c_[vec_lda * self.gamma, vec_bert]
            self.vec['LDA_BERT_FULL'] = vec_ldabert
            if not self.AE:
                self.AE = Autoencoder()
                print('Fitting Autoencoder ...')
                self.AE.fit(vec_ldabert)
                print('Fitting Autoencoder Done!')
            vec = self.AE.encoder.predict(vec_ldabert)
            return vec

    def fit(self, sentences, token_lists, method=None, m_clustering=None):
        """
        Fit the topic model for selected method given the preprocessed data
        :docs: list of documents, each doc is preprocessed as tokens
        :return:
        """
        # Default method
        if method is None:
            method = self.method
        # Default clustering method
        if m_clustering is None:
            m_clustering = KMeans

        # turn tokenized documents into a id <-> term dictionary
        if not self.dictionary:
            self.dictionary = corpora.Dictionary(token_lists)           

            #Remove unneccessary keywords from token list of corpus. First identify the keywords and then remove them from dictionary and respectively on the corpus vector            
            #rmv_words_corpus = ['provide','account','any','them','we','payment','or','company','dispute','you','request','by','an','our','your','will','consumer','due','send','which','over','receive','inform','show','never','add','their','please','owe','now','what','proof','state','our','amount','late','because','check','since','ask','never','apply','any','back','take','mine','notify','only','you','belong_me','can','proper','there','please','mail','know','sign','default','eviction','name','additional','debt','accuracy','behind','allege','subsection','regular','manager','instrument_bear','subparagraph','render','mother','content','accordance_with','landlord','door','data_furnisher','exact_same','description','second','completeness','shall','base_upon','absent','ratio','justify','sister','reasonably','operate','see_attachment','regularly','threatening','marry','stopped','bury','morning','pad','operator','displace','ship','mediate','account','request','our','inform','say','because','amount','contact','letter','need','insurance','so','any','due','will','credit','company','take','now','over','out','she','also','ask','date','bank','back','provide','only','give','up','just','add','change','increase','recent','shortage','how_much','partial','flood','renewal','periodic','reminder','apology','spread','transition','gap','rudely','flood_zone','midst','maturity','master_policy','state','document','file','any','what','request','late','can','who','account','case','modification','need','new','show','attorney','because','apply','ask','still','amount','never','only','record','will','date','you','your','correct','case_number','against','action','client','lie','trustee','attached','dual','recast','caliber','favor','relevant','entry','consequence','nominee','execution','character','days','clock','herein','united','pursuit','chat','notwithstanding','concrete','yours','willful','undo','inaction','patiently','hate','our','modification','company','request','tax','receive','say','now','an','try','dispute','address','since','so','out','by','ask','can','their','take','could','letter','what','them','proper','will','document','work','never','process','your','amount','your','up','back','over','account','there','you','due','at','sign','copy','send','free','file','inquiry','company','letter','credit','deny','balance','off','all','provide','foreclose','make','mail','can','into','which','investigation','contact','there','get','or','husband','number','fraud','personal','able','rate','since','want','way','access','submit','plan','month','form','assist','act','offer','county','initiate','rent','trust','site','lower','discover','live_at','understand','view','privacy','consent','employee','additionally','fill_out','best','residence','connect','obvious','pin','insert','conference','eligible','go','defer','back_onto','retrieve','annual','motion','expose','unwanted','carry','consequence','crime','taxpayer','vehicle','exempt','divide','racketeering','jumbo','imply','technical','reassess','bond','subpoena','hedge','gon_na','husband_die','star','scary','proven','bubble','spirit','excite','earner','margin','all','them','he','get','could','fee','so','dispute','send','letter','back','payment','which','but','or','only','request','modification','ask','try','now','give','our','process','can','there','never','state','again','say','you','there','by','receive','account','what','can','close','copy','need','submit','because','work','their','document','application','time','take','so','also','at','bureau','add','correct','file','these','company','please','help','transfer','list','after','want','still','keep','inquiry','lender','since','off','even','back','know','more','appraisal','representative','attach','him','per','return','order','his','nothing','access','none','deed','particular','packet','mother','pa','comment','age','bogus','identification','child','draft','real','waive','might','sent_certificate','hire','legitimately','second','treat','together','contractual','sum','stole','desist','world','mi','harassment','direction','imagine','counselor','corporate','caliber','priority','business_practice','sheriff','remodification','mi','reassign','arm','rush','combine','farm','faster','ethic','annuity','stall_tactic','declination','click','administer','offset','teacher','folder','unlike','dissatisfy','naturally','an','there','them','dispute','payment','our','you','get','all','can','need','receive','tell','balance','try','or','their','add','when','will','card','file','so','show','time','should','company','contact','because','only','but','would','score','service','account','escrow','close','can','request','amount','up','could','now','close','show','negative','document','help','one','new','state','check','amount','bill','modification','so','due','find','say','insurance','what','letter','will','because','date','company','there','out','should','could','never','still','ask','also','continue','file','issue','debt','off','original','refuse','vehicle','multiple','response','term','value','private','schedule','refund','power','release','post','reflect','county','average','press','front','rep','meanwhile','watch','karma','user','frustration','certificate','intentionally','accident','enforce','deduct','evidently','length','percent','green','time_fashion','newly','manual','ie','float','pro','anxious','frankly','assessed','pay','sever','tax','account','dispute','payment','close','by','verify','amount','incorrect','file','you','creditor','date','check','all','name','company','correct','when','open','document','there','so','add','false','now','which','need','an','can','our','show','delete','contact','time','use','call','but','show','regard','amount','phone','provide','out','company','will','another','previous','speak','correct','total','current','refund','satisfy','estimate','submission','understanding','town','stuff','enclose','manual','generic','photo','electronic','sincerely','advance','specify','rather_than','doc','pass','exemption','scan','card','hud','disburse','shortfall','wrongful','accumulate','minus','gift','realistic','contention','recur','column','girl','flip','sloppy','caught','continuance','buck','she','proper','company','house','now','file','say','can','ask','so','send','never','all','because','payment','any','sell','also','call','what','inform','take','will','score','never','their','still','which','contact','card','try','buy','over','about','letter','need','state','or','since','also','make','help','receive','show','through','by','add','her','know','but','get','loan','them','update','time','close','an','right','same','contact','start','he','place','would','about','name','own','result','address','resolve','him','down','his','title','low','away','happen','spoke','price','benefit','represent','firm','unit','got','messenger','duplex','man','neighbor','vacant','incident','per_month','incur','urgent','opinion','reinsert','extension','prefer','friend','borrow','perfect','sheet','street','water','brother','sometime','reward','differ','belonging','occupy','floor','yard','tree','ex_husband','marketing','dog','score_suffer','wide','liar','shape','predator','driveway','drug','redemption','cash_key','wow','heat','kickback','recorder','dispute','file','or','verify','our','say','them','by','request','account','these','item','bank','payment','consumer','amount','company','all','check','which','any','delete','state','so','could','because','send','add','bureau','you','but','letter','provide','modification','inaccurate','will','because','their','receive','also','over','document','fraud','never','document','investigation','ask','charge','escrow','date','now','she','still','back','state','request','take','can','list','there','give','new','interest','money','late','apply','refuse','issue','fund','law','right','insurance','immediately','even','court','correct','bill','principle','owe','put','must','proof','behind','furnish','policy','wrong','lien','source','figure','treatment','challenge','contractor','physical','any_kind','certification','certify','suspense','deduct','draw','relief','verifiable','deem','length','statue','adjudication','wit','disturb','entirely','nonexistent','eye','frozen','affair','inspector','auto','fay','revenue','storm','forced','teller','index','enough_cover','recalculate','payee','septic','occurrence','unhelpful','manually','spoken','our','letter','request','state','inform','process','document','account','provide','offer','bank','so','review','contact','modification','require','any','submit','there','back','close','approve','due','date','because','amount','say','application','need','also','report','borrower','add','return','response','attorney','advise','regard','within','term','must','per','delay','rule','guideline','appraiser','builder','accurately','tip','evade','periodic','chart','exclude','duration','directive','conditional','origin','noncompliance','rebate','remit','procure','reform','questionnaire','verifiable','accent','our','document','modification','you','inform','he','request','proper','say','foreclosure','account','bank','any','provide','ask','never','state','letter','process','there','so','will','date','house','can','because','file','give','know','about','short_sale','submit','deny','review','approve','complain','application','complete','your','notice','other','income','sale','personal','law','office','trial','appeal','decision','conduct','specialist','exhibit','forbearance','mediation','forge','eligibility','exercise','article','blank','hereby','format','foreign','harp','limitation','civil_code','commissioner','writer','log','facially_complete']
            rmv_words_corpus = ['inform','receive','mortgage','number','send','work','short','make','mother','account','contact','tell','proper','request','customer','wife','say','company','family','refinance','basically','date','value','unit','provide','check','job','record','continue','submit','roof','apply','complain','trust','new','remove','approve','land','try','increase','ask','husband','sign','high','right','correct','program','need','situation','term','chapter','line','search','bank','shortage','personal','cause','original','buy','trial','want','replace','case','loss','low','demand','negative','just','flood','representative','hardship','attempt','dispute','apartment','live','child','action','current','history','know','commitment','homeowner','response','speak','life','assignment','refuse','perform','analysis','owe','feel','complete','add','hire','county','additional','borrower','use','pull','divorce','option','require','start','place','write','question','copy','consumer','final']
            # assume the word 'b' is to be deleted, put its id in a variable
            self.del_ids = [k for k,v in self.dictionary.items() if v in rmv_words_corpus]

            # remove unwanted word ids from the dictionary in place
            self.dictionary.filter_tokens(bad_ids=self.del_ids)
            # convert tokenized documents into a document-term matrix
            self.corpus = [self.dictionary.doc2bow(text) for text in token_lists]
            gensim.corpora.MmCorpus.serialize('Mortgage_CorpusAll_Fit.mm', self.corpus)
            '''self.corpus = gensim.corpora.MmCorpus('Mortgage_CorpusAll.mm')
            # convert corpus to a dense array, transpose because by default documents would be columns
            self.np_corpus = gensim.matutils.corpus2dense(self.corpus, self.corpus.num_terms, num_docs=self.corpus.num_docs).T
            # delete columns for specified tokens, transpose back afterwards
            self.np_corpus = np.delete(self.np_corpus, self.del_ids, 1).T
            # convert array to corpus
            self.corpus = gensim.matutils.Dense2Corpus(self.np_corpus)'''
        
        ####################################################
        #### Getting ldamodel or vector representations ####
        ####################################################

        if method == 'LDA':
            if not self.ldamodel:
                print('Fitting LDA ...')
                self.ldamodel = gensim.models.ldamodel.LdaModel(self.corpus, num_topics=self.k, id2word=self.dictionary,
                                                                passes=20)
                print('Fitting LDA Done!')
        else:
            print('Clustering embeddings ...')
            self.cluster_model = m_clustering(self.k)
            self.vec[method] = self.vectorize(sentences, token_lists, method)
            self.cluster_model.fit(self.vec[method])
            print('Clustering embeddings. Done!')

    def predict(self, sentences, token_lists, out_of_sample=None):
        """
        Predict topics for new_documents
        """
        # Default as False
        out_of_sample = out_of_sample is not None

        if out_of_sample:
            corpus = [self.dictionary.doc2bow(text) for text in token_lists]
            if self.method != 'LDA':
                vec = self.vectorize(sentences, token_lists)
                print(vec)
        else:
            corpus = corpus
            vec = self.vec.get(self.method, None)

        if self.method == "LDA":
            lbs = np.array(list(map(lambda x: sorted(self.ldamodel.get_document_topics(x),
                                                     key=lambda x: x[1], reverse=True)[0][0],
                                    corpus)))
        else:
            lbs = self.cluster_model.predict(vec)
        return lbs

## Training

In [ ]:
#from model import *
#from utils import *

import pandas as pd
import pickle
import matplotlib.pyplot as plt
# Import spacy for lemmatization
import spacy

import warnings
warnings.filterwarnings('ignore', category=Warning)

import argparse

#def model(): #:if __name__ == '__main__':

def main():
    
    
    #method = "TFIDF"
    #method = "LDA"
    #method = "BERT"    
    method = "LDA_BERT"
    samp_size = 10000
    ntopic = 7
    
    #parser = argparse.ArgumentParser(description='contextual_topic_identification tm_test:1.0')

    #parser.add_argument('--fpath', default='/kaggle/working/train.csv')
    #parser.add_argument('--ntopic', default=10,)
    #parser.add_argument('--method', default='TFIDF')
    #parser.add_argument('--samp_size', default=20500)
    
    #args = parser.parse_args()
    complain_data= pd.read_excel(r'/content/drive/My Drive/CFPB LDA/LDA Output Document to topic.xlsx')
    data_train = complain_data['Cleansed_Text'].head(10)   
    sentences, token_lists, idx_in = preprocess(data_train, samp_size=samp_size)
    # Define the topic model object
    #tm = Topic_Model(k = 10), method = TFIDF)
    tm = Topic_Model(k = ntopic, method = method)
    # Fit the topic model by chosen method
    tm.fit(sentences, token_lists)

    print('Coherence:', get_coherence(tm, token_lists, 'c_v'))
    print('Silhouette Score:', get_silhouette(tm))
    # visualize and save img
    visualize(tm)
    for i in range(tm.k):
        get_wordcloud(tm, token_lists, i)   

In [ ]:
main()

In [ ]:
tm.predict(sentences_test, token_lists_test, corpus)

In [ ]:
complain_data_test= pd.read_csv(r'/content/drive/My Drive/CFPB LDA/Mortgage_complaints_text_train.csv')
print(complain_data_test.columns)
data_test = complain_data_test['Complain_text']
samp_size = 10000    
sentences_test, token_lists_test, idx_in_test = preprocess(data_test, samp_size=samp_size)

In [ ]:
# Import spacy for lemmatization
import spacy
# Create a list of all words in the dataset
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
        
data_words = list(sent_to_words(data_test))

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]
def lemmatization(texts, 
                  allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc 
                          if token.pos_ in allowed_postags])
    return texts_out

# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words , 
                  allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])    

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)
# Create Corpus
texts = data_lemmatized
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [ ]:
lbs = np.array(list(map(lambda x: sorted(ldamodel.get_document_topics(x),key=lambda x: x[1], reverse=True)[0][0],corpus)))

In [ ]:
ntopic = 11
method = "LDA"
tm = Topic_Model(k = ntopic, method = method)
tm.predict(sentences_test, token_lists_test)

In [ ]:
corpus = [dictionary.doc2bow(text) for text in token_lists_test]
lbs = np.array(list(map(lambda x: sorted(ldamodel.get_document_topics(x),key=lambda x: x[1], reverse=True)[0][0],corpus)))

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(token_lists_test)
# Create Corpus
texts = sentences_test
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

### TODO:

* Implement models.ldamulticore – parallelized Latent Dirichlet Allocation using all CPU cores to parallelize and speed up model training.
* Switch from BERT/RoBERTa to SciBERT, BART, and or other models. 